In [43]:
import cv2
import rosbag
import pandas as pd
import numpy as np
import re
from os import listdir
from os.path import isfile, join

In [44]:
bag = rosbag.Bag('../bags/trackMe.bag')

In [45]:
print bag

path:        ../bags/trackMe.bag
version:     2.0
duration:    29.8s
start:       Apr 10 2017 13:55:01.49 (1491825301.49)
end:         Apr 10 2017 13:55:31.28 (1491825331.28)
size:        527.9 MB
messages:    5346
compression: none [356/356 chunks]
types:       opt_msgs/DetectionArray [2be460a0a39ff0f809695e59da9d469e]
             opt_msgs/TrackArray     [4106b2da5c66b9f239aada7912e67b8f]
             sensor_msgs/Image       [060021388200f6f0f447d0fcd9c64743]
             tf2_msgs/TFMessage      [94810edda583a504dfda3829e70d7eec]
topics:      /Kinect_Laurene/rgb_lowres/image    355 msgs @  11.8 Hz : sensor_msgs/Image      
             /tf                                3567 msgs @ 140.2 Hz : tf2_msgs/TFMessage      (12 connections)
             /tracker/tracks                     712 msgs @  20.7 Hz : opt_msgs/TrackArray    
             detector/detections                 712 msgs @  22.6 Hz : opt_msgs/DetectionArray (2 connections)


---
Need to match a frame for each not empty detection msg <br />
How to store this matching? Put everything in CSV format. <br />
So need to create 2 dataframes: one for detection and one for frame <br />
From detection dataframe, need create a none empty detection dataframe (one column "time" and one column "msg")


---

In [46]:
# open the detection text file
detectionFile = open('/home/sabrine/notebook/messages/detectMeL.txt', 'r') 

In [47]:
# Create a array with all the messages
detection = detectionFile.read().split('header: \n')[1:]

In [48]:
# Extract all the information for each messages
seq = []
secs = []
nsecs = []
frame_id = []
detections = []
intrinsic_matrix = []
image_type = []
for i in range(len(detection)):
    seq.append(re.search('[0-9]+',re.search('seq: [0-9]+', detection[i]).group(0)).group(0))
    secs.append(re.search('[0-9]+',re.search('stamp: \n    secs: [0-9]+', detection[i]).group(0)).group(0))
    nsecs.append(re.search('[0-9]+',re.search('nsecs:\s+[0-9]+', detection[i]).group(0)).group(0))
    frame_id.append(re.search('/[A-z]+' ,re.search('frame_id: /[A-z]+', detection[i]).group()).group())
    detections.append(re.search('detections: (?s).*\intrinsic_matrix', detection[i]).group().replace('detections: ', '').replace('intrinsic_matrix', ''))
    intrinsic_matrix.append(re.search('intrinsic_matrix: \[.*\]', detection[i]).group().replace('intrinsic_matrix: ', ''))

In [49]:
# Create an empty df with the right columns
det = pd.DataFrame(columns=['seq', 'secs', 'nsecs', 'frame_id', 'detections', 'intrinsic_matrix'])

In [50]:
# Then fill in the empty df with the info extracted before
det.seq = seq
det.secs = secs
det.nsecs = nsecs
det.frame_id = frame_id
det.detections = detections
det.intrinsic_matrix = intrinsic_matrix

In [51]:
# check the result
det.head()

,seq,secs,nsecs,frame_id,detections,intrinsic_matrix
0,2535,1491825301,379490000,/Kinect_Nuc_ir_optical_frame,[]\n,"[361.6513366699219, 0.0, 260.69268798828125, 0..."
1,1720,1491825301,218655000,/Kinect_Laurene_rgb_optical_frame,[]\n,"[528.6603393554688, 0.0, 476.2093811035156, 0...."
2,2536,1491825301,447161000,/Kinect_Nuc_ir_optical_frame,[]\n,"[361.6513366699219, 0.0, 260.69268798828125, 0..."
3,1721,1491825301,281000000,/Kinect_Laurene_rgb_optical_frame,[]\n,"[528.6603393554688, 0.0, 476.2093811035156, 0...."
4,2537,1491825301,509378000,/Kinect_Nuc_ir_optical_frame,[]\n,"[361.6513366699219, 0.0, 260.69268798828125, 0..."


In [52]:
det.nsecs()

TypeError: 'Series' object is not callable

In [53]:
# Create a new df with only the none empty detection field
det2 = det[det['detections'].map(len) > 5]

In [54]:
#Check result
det2.head()

,seq,secs,nsecs,frame_id,detections,intrinsic_matrix
7,2539,1491825301,642750000,/Kinect_Nuc_ir_optical_frame,\n - \n box_3D: \n p1: \n x: 4...,"[361.6513366699219, 0.0, 260.69268798828125, 0..."
11,2541,1491825301,775816000,/Kinect_Nuc_ir_optical_frame,\n - \n box_3D: \n p1: \n x: -...,"[361.6513366699219, 0.0, 260.69268798828125, 0..."
13,2543,1491825301,906281000,/Kinect_Nuc_ir_optical_frame,\n - \n box_3D: \n p1: \n x: -...,"[361.6513366699219, 0.0, 260.69268798828125, 0..."
16,2545,1491825302,42033000,/Kinect_Nuc_ir_optical_frame,\n - \n box_3D: \n p1: \n x: -...,"[361.6513366699219, 0.0, 260.69268798828125, 0..."
20,2548,1491825302,249109000,/Kinect_Nuc_ir_optical_frame,\n - \n box_3D: \n p1: \n x: -...,"[361.6513366699219, 0.0, 260.69268798828125, 0..."


---
Now need to order the detection messages df by secs and then nsces

In [55]:
# Order the detection messages by secs and nsecs in a new df
det3 = det2.sort_values(by=['secs', 'nsecs'])
det3.index = range(0,len(det3))

In [56]:
# check result (see 2 last lines of the head)
det3.head(-20)

,seq,secs,nsecs,frame_id,detections,intrinsic_matrix
0,2539,1491825301,642750000,/Kinect_Nuc_ir_optical_frame,\n - \n box_3D: \n p1: \n x: 4...,"[361.6513366699219, 0.0, 260.69268798828125, 0..."
1,2541,1491825301,775816000,/Kinect_Nuc_ir_optical_frame,\n - \n box_3D: \n p1: \n x: -...,"[361.6513366699219, 0.0, 260.69268798828125, 0..."
2,2543,1491825301,906281000,/Kinect_Nuc_ir_optical_frame,\n - \n box_3D: \n p1: \n x: -...,"[361.6513366699219, 0.0, 260.69268798828125, 0..."
3,2548,1491825302,249109000,/Kinect_Nuc_ir_optical_frame,\n - \n box_3D: \n p1: \n x: -...,"[361.6513366699219, 0.0, 260.69268798828125, 0..."
4,2545,1491825302,42033000,/Kinect_Nuc_ir_optical_frame,\n - \n box_3D: \n p1: \n x: -...,"[361.6513366699219, 0.0, 260.69268798828125, 0..."
5,2553,1491825302,576870000,/Kinect_Nuc_ir_optical_frame,\n - \n box_3D: \n p1: \n x: -...,"[361.6513366699219, 0.0, 260.69268798828125, 0..."
6,2555,1491825302,706769000,/Kinect_Nuc_ir_optical_frame,\n - \n box_3D: \n p1: \n x: -...,"[361.6513366699219, 0.0, 260.69268798828125, 0..."
7,2559,1491825302,979154000,/Kinect_Nuc_ir_optical_frame,\n - \n box_3D: \n p1: \n x: -...,"[361.6513366699219, 0.0, 260.69268798828125, 0..."
8,2561,1491825303,109898000,/Kinect_Nuc_ir_optical_frame,\n - \n box_3D: \n p1: \n x: -...,"[361.6513366699219, 0.0, 260.69268798828125, 0..."
9,2562,1491825303,177423000,/Kinect_Nuc_ir_optical_frame,\n - \n box_3D: \n p1: \n x: -...,"[361.6513366699219, 0.0, 260.69268798828125, 0..."


Now need to create an array of time of frame creation <br />
Then need to math each detection message to a frame by selecting the closest one but younger.

In [57]:
# List all file from frames folder
mypath = "/home/sabrine/notebook/framesRGB"
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [58]:
# Extract time from files' name
time_ = []
for t in onlyfiles:
    time_.append(int(re.search('^[0-9]*', t).group()))

In [59]:
# Sort the list of time in order help the future match
sorted_ = sorted(time_, key=int) 

In [60]:
# Recreate the time of the detection in order to compare it to the frames' file name
timeDetec = (det3.secs + det3.nsecs).apply(pd.to_numeric).values.T.tolist()

In [61]:
# Use the fact that the frames'time list is ordered to easly find the closest match with a detection
def closestYounger(detectionTime, list_frameTime):
    closestTime = "error"
    index = "none"
    for i, frameTime in enumerate(list_frameTime):
        if frameTime < detectionTime and (detectionTime-frameTime) < 10000000000:
            closestTime = frameTime
            index = i
        else:
            break
    return index, closestTime

In [62]:
def synchronizer(detectionList, frameList, dictionary):
    if len(frameList) != 0 and len(detectionList) != 0:
        index, time = closestYounger(detectionList[0], frameList)
        dictionary[detectionList[0]] = time
        print("detection time : ", detectionList[0])
        print("time matched : ", time)
        #print("fameList : ", frameList)
        if len(detectionList) > 0 and len(frameList) > 0:
            print("detection : ", len(detectionList))
            print("frame : ", len(frameList))
            if index != 'none':
                shorterFrameTimeList = frameList[:index] + frameList[index+1 :]
                synchronizer(detectionList[1:], shorterFrameTimeList, dictionary)
            else:
                synchronizer(detectionList[1:], frameList, dictionary)                
    return dictionary

In [63]:
len(sorted_)

355

In [64]:
synchronizer(timeDetec, sorted_, {})

('detection time : ', 1491825301642750000)
('time matched : ', 1491825301607465067)
('detection : ', 474)
('frame : ', 355)
('detection time : ', 1491825301775816000)
('time matched : ', 1491825301698317405)
('detection : ', 473)
('frame : ', 354)
('detection time : ', 1491825301906281000)
('time matched : ', 1491825301896877071)
('detection : ', 472)
('frame : ', 353)
('detection time : ', 1491825302249109000)
('time matched : ', 1491825302203631210)
('detection : ', 471)
('frame : ', 352)
('detection time : ', 149182530242033000)
('time matched : ', 'error')
('detection : ', 470)
('frame : ', 351)
('detection time : ', 1491825302576870000)
('time matched : ', 1491825302572655847)
('detection : ', 469)
('frame : ', 351)
('detection time : ', 1491825302706769000)
('time matched : ', 1491825302614977091)
('detection : ', 468)
('frame : ', 350)
('detection time : ', 1491825302979154000)
('time matched : ', 1491825302906372191)
('detection : ', 467)
('frame : ', 349)
('detection time : ',

{14918253106828000: 'error',
 14918253145799000: 'error',
 14918253272268000: 'error',
 14918253298341000: 'error',
 149182530242033000: 'error',
 149182530457471000: 'error',
 149182530515444000: 'error',
 149182530612060000: 'error',
 149182530679958000: 'error',
 149182530780186000: 'error',
 149182530945407000: 'error',
 149182531079550000: 'error',
 149182531112560000: 'error',
 149182531129840000: 'error',
 149182531178321000: 'error',
 149182531212155000: 'error',
 149182531279295000: 'error',
 149182531312521000: 'error',
 149182531320807000: 'error',
 149182531374380000: 'error',
 149182531449131000: 'error',
 149182531478324000: 'error',
 149182531512768000: 'error',
 149182531578986000: 'error',
 149182531610430000: 'error',
 149182531677490000: 'error',
 149182531745785000: 'error',
 149182531836521000: 'error',
 149182531840740000: 'error',
 149182531946069000: 'error',
 149182531993714000: 'error',
 149182532013337000: 'error',
 149182532079818000: 'error',
 1491825321124

In [65]:
1491825301642750000-1491825301551694486

91055514